In [54]:
import pandas as pd

In [59]:
def read_dataframes():
    """
        In this function,  we are are reading the required csv files from the dataset into dataframes.
        :return: We return the Ibm_df, adult_df as dataframes after reading the csv files.
    """
    Ibm_df = pd.read_csv('C:/Users/alka9/IBM_HR_Dataset.csv', delimiter=',', encoding='UTF-8')
    ad_df = pd.read_csv('C:/Users/alka9/adult.csv', delimiter=',', header = None, encoding='UTF-8')
    ad_df.columns = ['Age', 'JobType', 'EmpID',
                          'EducationLevel', 'Level', 'MaritalStatus',
                          'JobPosition', 'MaritalStatus_Desc', 'Race',
                          'Gender', 'Column_1', 'Column_2',
                          'Column_3', 'Location', 'ExpectedSalary']
    adult_df = ad_df[['Age', 'JobType', 'EducationLevel', 'Level', 'JobPosition', 'MaritalStatus', 'Location', 'Gender', 'ExpectedSalary']].sort_values(by='Age', ascending=True)
    return Ibm_df, adult_df

In [58]:
def filter_data(adult_df):
    """
        In this function, we are are choosing adult dataset based on Job type, location and age.
        :return: We return the ad_df_private_US after filtering.
    """
    adult_df['JobType'] = adult_df['JobType'].str.strip()
    adult_df['Location'] = adult_df['Location'].str.strip()
    adult_df['Gender'] = adult_df['Gender'].str.strip()
    adult_df['JobPosition'] = adult_df['JobPosition'].str.strip()
    adult_df['EducationLevel'] = adult_df['EducationLevel'].str.strip()
    adult_df['ExpectedSalary'] = adult_df['ExpectedSalary'].str.strip()
    ad_df_private = adult_df[adult_df['JobType'] == 'Private']
    ad_df_private_US = ad_df_private[ad_df_private['Location'] == 'United-States']
    ad_df_private_US = ad_df_private_US[ad_df_private_US['Age'] > 17]
    ad_df_private_US = ad_df_private_US[ad_df_private_US['Age'] <= 60] 
    return ad_df_private_US

In [57]:
def club_similar_values(adult_df):
    """
        In this function, we are are replacing Job position values with common positions in adult dataset so that the job positions remain common accross csv's.
        :return: We return the ad_df_private_US after replacement.
    """
    ad_df_private_US.loc[ad_df_private_US.JobPosition == "Adm-clerical", 'Education Field'] = "Human Resources"
    ad_df_private_US.loc[ad_df_private_US.JobPosition == "Farming-fishing", 'Education Field'] = "Life Sciences"
    ad_df_private_US.loc[(ad_df_private_US.JobPosition == "Machine-op-inspct") | (ad_df_private_US.JobPosition == "Tech-support"), 'Education Field'] = "Technical Degree"
    ad_df_private_US.loc[(ad_df_private_US.JobPosition == "Other-service") | (ad_df_private_US.JobPosition == "Transport-moving") | (ad_df_private_US.JobPosition == "Handlers-cleaners"), 'Education Field'] = "Other"
    ad_df_private_US.loc[(ad_df_private_US.JobPosition == "Protective-serv") | (ad_df_private_US.JobPosition == "Prof-specialty"), 'Education Field'] = "Medical"
    ad_df_private_US.loc[ad_df_private_US.JobPosition == "Sales", 'Education Field'] = "Management"

    ad_df_private_US.loc[(ad_df_private_US.EducationLevel == "10th") | (ad_df_private_US.EducationLevel == "11th") | (ad_df_private_US.EducationLevel == "12th") | (ad_df_private_US.EducationLevel == "1st-4th") | (ad_df_private_US.EducationLevel == "5th-6th")
                         | (ad_df_private_US.EducationLevel == "7th-8th") | (ad_df_private_US.EducationLevel == "9th"), 'Education'] = 2
    ad_df_private_US.loc[(ad_df_private_US.EducationLevel == "Assoc-acdm") | (ad_df_private_US.EducationLevel == "Assoc-voc") | (ad_df_private_US.EducationLevel == "Some-college") | (ad_df_private_US.EducationLevel == "HS-grad"), 'Education'] = 2
    ad_df_private_US.loc[ad_df_private_US.EducationLevel == "Bachelors", 'Education'] = 3
    ad_df_private_US.loc[(ad_df_private_US.EducationLevel == "Masters") | (ad_df_private_US.EducationLevel == "Prof-school"), 'Education'] = 4
    ad_df_private_US.loc[ad_df_private_US.EducationLevel == "Doctorate", 'Education'] = 5

    ad_df_private_US
    return ad_df_private_US

In [56]:
def probable_expected_salary(ad_df_private_US):
    """
        In this function, we are finding probable expected salary values based on age group degree, education field and gender.
        For probable expected value between 0 to 50 : Expected Salary is less than or equal to 50k
        For probable expected value greater than 50 : Expected Salary is less greater than 50k
        :return: We return the ad_dataframe after finding probable expected value.
    """
    ad_df_private_US = ad_df_private_US.dropna(subset=['Education Field'])
    ad_df_private_US.loc[ad_df_private_US.ExpectedSalary == "<=50K", 'Salary Expected'] = 0
    ad_df_private_US.loc[ad_df_private_US.ExpectedSalary == ">50K", 'Salary Expected'] = 100
    ad_dataframe = pd.DataFrame(ad_df_private_US.groupby(['Age','Education','Education Field', 'Gender'])['Probable Salary Value'].mean())
    return ad_dataframe

In [55]:
def merge_datasets(Ibm_df, ad_dataframe):
    """
        In this function, we are merging both the datasets based on Age, EducationField, Gender and Education.
        :return: We return the sorted_dataframe after merging.
    """
    Ibm_df['EducationField'] = Ibm_df['EducationField'].str.strip()
    merged_dataframe = pd.merge(Ibm_df, ad_dataframe,  how='inner', left_on=['Age','EducationField', 'Gender', 'Education'], right_on = ['Age','Education Field', 'Gender', 'Education'])
    sorted_dataframe = merged_dataframe.sort_values(by='Age', ascending=True)
    return sorted_dataframe

## Hypothesis 1 : If the salary of the employee is less than expected, it leads to attrition

In [ ]:
def split_dataframe_for_analysis_1(merged_dataframe):
    """
        In this function, we are splitting the dataframe based on expected salary .
        :return: We return the sorted_dataframe after merging.
    """
    merged_dataframe['Salary Earned'] = merged_dataframe.MonthlyIncome *12
    salary_split = merged_dataframe[merged_dataframe['Attrition'] == 'Yes']
    salary_less = salary_greater[salary_greater['Salary Expected'] <= 50.0]
    salary_greater = salary_greater[salary_greater['Salary Expected'] >  50.0]
    return salary_less, salary_greater

In [60]:
if __name__ == '__main__':
    Ibm_df, adult_df = read_dataframes()
    ad_df_private_US = filter_data(adult_df)
    ad_df_private_US = club_similar_values(ad_df_private_US)
    ad_dataframe = probable_expected_salary(ad_df_private_US)
    merged_dataframe = merge_datasets(Ibm_df, ad_dataframe)
    salary_less, salary_greater = split_dataframe_for_analysis_1(merged_dataframe)
    salary_attrition_analysis(salary_less)

In [61]:
merged_dataframe

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Salary Expected
544,18,No,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,...,80,0,0,2,3,0,0,0,0,0.000000
659,18,Yes,Travel_Frequently,544,Sales,3,2,Medical,1,1624,...,80,0,0,2,4,0,0,0,0,0.000000
423,19,Yes,Travel_Rarely,489,Human Resources,2,2,Technical Degree,1,566,...,80,0,1,3,4,1,0,0,0,0.000000
585,19,No,Travel_Rarely,645,Research & Development,9,2,Life Sciences,1,1193,...,80,0,1,4,3,1,1,0,0,0.000000
595,19,Yes,Non-Travel,504,Research & Development,10,3,Medical,1,1248,...,80,0,1,2,4,1,1,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,58,No,Travel_Rarely,682,Sales,10,4,Medical,1,131,...,80,0,38,1,2,37,10,1,8,66.666667
552,59,No,Travel_Rarely,1089,Sales,1,2,Technical Degree,1,1048,...,80,1,14,1,1,6,4,0,4,33.333333
273,59,No,Travel_Rarely,818,Human Resources,6,2,Medical,1,321,...,80,0,7,2,2,2,2,2,2,50.000000
601,59,No,Travel_Rarely,1429,Research & Development,18,4,Medical,1,1283,...,80,0,25,6,2,9,7,5,4,16.666667
